# Pandas Tutorial 
Pandas is a python library commonly used for analyzing, filtering and manipulating data. It works particularly well with any tabular ("table like") data, including the data sets that we will be using during Week 1. In this notebook, we will explore how pandas can be used to read, visualize, and modify data sets. For further information, see the pandas documentation website here: https://pandas.pydata.org/docs/index.html

## Pima Diabetes Dataset

This notebook tutorial and most of the other tutorial notebooks for this week will use data from the Pima Diabetes Dataset. The set we will use can be accessed with the following direct link:

https://raw.githubusercontent.com/BeaverWorksMedlytics2020/Data_Public/master/NotebookExampleData/Week1/diabetes.csv

### About the data:

All patients here are females at least 21 years old of Pima Native American heritage.

- **Pregnancies**: Number of times pregnant
- **Glucose**: Plasma glucose concentration 2 hours in an oral glucose tolerance test
- **BloodPressure**: Diastolic blood pressure (mm Hg)
- **SkinThickness**: Triceps skin fold thickness (mm)
- **Insulin**: 2-Hour serum insulin (mu U/ml)
- **BMI**: Body mass index (weight in kg/(height in m)^2)
- **DiabetesPedigreeFunction**: A function that scores the likelihood of diabetes based on family history.
- **Age**: Age (years)
- **Outcome**: 1 (has diabetes) or 0 (no diabetes)

## Reading CSV files with pandas

Pandas can read data directly from a url. There are a number of options associated with the **read_csv** function, but for now we will just pass a list of the column labels, since our CSV does not contain them. Without this option, the first row of data would be interpreted as labels.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/BeaverWorksMedlytics2020/Data_Public/master/NotebookExampleData/Week1/diabetes.csv"

"""
      'preg': number of pregnancies  
      'plas': plasma glucose concentration 
      'pres': blood pressure 
      'skin': skin thickness
      'test': Insluin
      'mass': BMI
      'pedi': diabetes pedigree function
      'age': age
      'class': '0' means does not have diabetes and '1' means has diabetes
"""

# Define explicit list of column headers to pass to pandas
our_names = ['preg','plas','pres','skin','test','mass','pedi','age','class']
data = pd.read_csv(url, names=our_names)

Now that we've imported the data, let's take a closer look at the python object that was created by pandas. First, what is the object's type?

In [ ]:
type(data)

DataFrame is a very general structure for 2-dimensional tabular data. It contains column labels, is size-mutable, and may contain heterogeneous data (this means data may be missing, or of mixed types). There are a number of ways to get information about a DataFrame. Each of the following code segments will demonstrate one such way.

In [ ]:
# Show the first 10 entries in the DataFrame (from the "head")
data.head(10)

In [ ]:
# Get the dimensionality of the DataFrame (number of rows, columns)
data.shape

In [ ]:
# Find the number of entries for each column
data.count()

In [ ]:
# Create a string that describes the shape of the dataset
"There are " + str(data.shape[0]) + " people in the study with " + str(data.shape[1]) + " variables each "

In [ ]:
# Generate a table of statistical information describing the dataset
data.describe()

Try using tab complete to see other functions and variables associated with DataFrames, or explore the API documentation:

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

## Slicing and Dicing

Slicing and dicing are technical terms for subselecting parts of a multi-dimensional dataset. To **slice** a dataset is to choose a value for one dimension, and take all data that matches. This results in a subset that is one dimension smaller. For example, in our dataset we could slice a single row or column. To **dice** data is to choose ranges in multiple dimensions, creating a more arbitrary subset.

To take a simple example, selecting a value from a python list by index is technically a slice (1-D to 0-D). Selecting a range of indices would be a dice. Pandas allows for similar data access, either by index or label. Here are some examples:
<br></br>

**iloc**: Indexing with integers. Accepts single values [ i ], lists [ i, j ], or ranges [ i : j : k ].

In [ ]:
# Single-row slice
data.iloc[3]

In [ ]:
# Dicing the first three rows
data.iloc[0:3]  # NOTE: equivalent to data.iloc[0:3:1]

In [ ]:
# 2-D dice of the first 3 rows, but only the last two columns
data.iloc[0:3, 7:]  # NOTE: equivalent to data.iloc[0:3:1, 7:9:1]

**loc**: Indexing with column labels, which are strings for columns and integers for rows. Accepts single values [ 'pres' ], lists [ 'pres', 'class' ], or "label slices" [ 'pres' : 'class' ]. Note that the "label slice" is not a range, so the terminal value will be included. This is particularly confusing with row labels, because the syntax for a slice will resemble an integer range, as shown below.

In [ ]:
# Dice the first three rows, with data in the columns from 'pres' to 'class'
data.loc[0:2, 'pres':'class'] # NOTE: 0:2 is a slice, not a range, and includes index 2

## Chained Indexing/Masking

Now we know how to subselect data, but that's not very useful unless we know what data we want to subselect. One solution is to create a data **mask**. In this context, the mask is a list of indices corresponding to data that meets certain criteria. For example, we can create a mask for all data for people with an age greater than 39. Doing so is quite simple in pandas:

In [ ]:
# Create a pandas series "mask" for rows with an age greater than 39
data_age_boolean = data.age > 39
# Show the created boolean series
data_age_boolean.head(20)

The created object is a series of True/False values for each row. Let's double check the first three values for age.

In [ ]:
# Show the first three age values. Only value 0 is greater than 39.
data['age'].head(3)

In [ ]:
# Show the first three values in the boolean mask. Only value 0 should be TRUE.
data_age_boolean.head(3)

The series object can be used to access a subset of the data. This is effectively filtering the entire dataset to return only entries with an age greater than 39.

In [ ]:
# Subselect data that meets the condition set by data_age_boolean
filtered_dataframe_by_age = data[data_age_boolean]
# Display the resulting filtered data
filtered_dataframe_by_age

Notice that while the filtered output is only 207 rows, the row label goes up to 766. This is because the subselected rows have kept their original labels. Row "766" is the 207th row in our new structure. This can be important if we want to refer back to the original dataset.

Often we will want to apply more than one filter to our data. Fortunately, since the masks are just booleans, it is intuitive and easy to combine masks. For example, if we want to combine our age filter with a second filter, we can use a logical "and" operator (&) like this:

In [ ]:
# Subselect data with age greater than 39 and more than 2 pregnancies
filtered_dataframe_age_preg = data[data_age_boolean & (data['preg'] > 2)]
# Display the filtered data
filtered_dataframe_age_preg

Now we've reduced our subset to just 172 entries. Also, while the original row labels are still maintained, row "766" has been filtered out!

As a final note, you can also combine filters with the "or" operator (|). We would expect this to produce a subset of equal or larger size than either filter on its own.

In [ ]:
# Subselect data with age greater than 39 OR more than 2 pregnancies
filtered_dataframe_age_preg = data[data_age_boolean | (data['preg'] > 2)]
# Display the filtered data
filtered_dataframe_age_preg

You can also sort the values in a dataframe by ascending or descending order. This can be useful for quickly viewing, for example, all of the subjects who have had more than 12 pregnancies.

In [ ]:
sample = filtered_dataframe_age_preg.sort_values(by='preg', ascending=False, na_position='first') #descending order
sample #ordered based on "preg"

# Dealing with missing data

Not all datasets are created whole. Survey respondents may accidentally skip a question, some lab equipement could malfunction, or the handwriting for an answer may be illegible to the people entering the data into a computer. Anyway it could happen, and missing data is annoying!

There are a few ways to go about fixing those fields in your data, each with their own advantages and disadvantages. We'll continue using the Pima diabetes dataset to demonstrate.

In [ ]:
data.head(15)

Now if you look into the dataframe you'll see some odd, or rather **missing**, values. We can tell that these are missing values from knowing what the columns stand for; for example it wouldn't make sense for someone to have a blood pressure or BMI of 0 right? 

In [ ]:
# Looking at the columns with 0s
data.describe()

You can see from the 4th row showing minimum values that 'preg', 'plas', 'pres', 'skin', 'test', 'mass', and 'class' have 0s. On the other hand don't forget that for columns like number of pregnancies and class, a 0 is perfectly valid.

_This is why it's important that you know what type of data you're handling!_

To summarize, the columns with invalid 0s (missing data) are:
- plas
- pres
- skin
- test
- mass

## Replacing with a constant

A lot of times when data is missing the cell is simply left blank or filled with NaNs or constant; in our case **_they have already been replaced with the constant 0_**. However as said before we want to keep the 0s in columns representing pregnancies and class so it'd be best to somehow differentiate these missing and valid values.

NaN stands for "Not a Number" and is usually used for missing entries in a dataframe. Let's go ahead and replace the invalid 0s with NaNs.

In [ ]:
invalid = ['plas', 'pres', 'skin', 'test', 'mass']

for i in invalid:
    data[i].replace(to_replace=0, value=np.nan, inplace=True)

data.head(15)

**Pros**
- easiest and quickest way to deal with missing values


**Cons**
- can introduce bias and skew data

## Dropping rows

One of the next easiest ways to deal with NaNs is simply deleting rows with missing information. Luckily, pandas already has a beautiful function just for that purpose.

In [ ]:
data_row = data.dropna(axis=0).reset_index(drop=True) # axis: 0=row, 1=column

data_row.head(15)

But be cautious with dropping rows, depending on how many rows had NaNs and the size of your data. You may inadvertently drastically decrease the sample size. As you can see below, we lost about half of our data:

In [ ]:
print("Original size: ", data.shape)
print("With rows dropped: ", data_row.shape)

**Pros**
- again, easy and quick to apply to data

**Cons**
- reduces sample size

## Dropping the variable (column)

If the majority of the column were filled with NaNs (ie >70% but it's up to you to determine your own rule of thumb) and thus not very important in predicting the targeted label, dropping the variable makes sense. Although overall, it would be unwise to drop a variable just because it has NaNs; as you can see in our data that would mean dropping 5 of our 9 columns!

For an example's sake, let's drop the column containing the most NaNs.

In [ ]:
for i in invalid:
    count = data[i].isna().sum()
    print(i, ": ", count, "NaNs, ", ((count/(len(data.index))) * 100), "%")

In [ ]:
data_col = data.drop(['test'], axis=1)

data_col.head()

**Pros**
- can clean up unnecessary data and speed up processing

**Cons**
- reduces data and number of potential features

## Replacing with the mean/median

We can also deal with missing values by replacing them with the mean or median if the column's numerical. Like dropping NaNs, filling NaNs with a constant value can be done with a single function. 

In [ ]:
for i in invalid:
    data[i].fillna(data[i].mean(), inplace=True) #use data[i].median() for median

data.head()

**Pros**
- easy and straightforward (calculate, find, & replace)

**Cons**
- skews data to be biased towards the center, underestimating the variance in our data 